In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
import json
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error as rmse
import pickle
import numpy as np

In [ ]:
df = pd.read_csv('data/train_data.csv')
#df['xc_px'], df['yc_px'] = df['yc_px'], df['xc_px'] (não rodar isso)
#df['xc'], df['yc'] = df['yc'], df['xc'] (não rodar isso) (deixei para dxar registrado)
#df.to_csv(f"data/train_data.csv", index=False) 

In [29]:
df = df.dropna()

min_max_scalers = {}

#scaler = MinMaxScaler()
for column in df.columns:
    if column in  ['xc_px', 'yc_px', 'xc', 'yc']:
        min_max_scalers[column] = MinMaxScaler() #(df[column].min(), df[column].max())
        df[column] = min_max_scalers[column].fit_transform(df[[column]])
        with open('scalers/'+column+'.pkl', 'wb') as f:
            pickle.dump(min_max_scalers[column], f)

    
#with open('data/min_max_scalers.json', 'w') as f:
#    json.dump(min_max_scalers, f)

df.head()

,x,y,z,t0,t1,t2,t3,xc_px,yc_px,xc,yc,diagonal
7,0.16,-0.19,0.12,30.100908,33.948155,75.041009,41.092854,0.591045,0.012174,0.591045,0.987826,73.389373
8,0.17,-0.18,0.12,33.363423,34.952635,76.655631,41.702995,0.635821,0.031304,0.635821,0.968696,72.801099
9,0.18,-0.17,0.12,36.636577,34.952635,76.655631,41.702995,0.689552,0.057391,0.689552,0.942609,72.780492
10,0.19,-0.16,0.12,39.899092,33.948155,75.041009,41.092854,0.728358,0.081739,0.728358,0.918261,72.835431
11,0.20,-0.15,0.12,43.130102,31.753588,71.469230,39.715641,0.779104,0.104348,0.779104,0.895652,75.239617


## 1. Utilizando o Mean Squared Error

In [3]:
import torch
import torch.nn as nn

In [86]:
def perc_pos(value, perc):
    return int(value*perc)/100

In [ ]:
input_data = torch.Tensor()
output_data = torch.Tensor()

for i in range(len(df)):
     row = df.iloc[i]
     x_fin, y_fin = row['x']*100, row['y']*100
     pos_hist = torch.Tensor([[314,134,0.035,0]])
     for p in [0.25, 0.5, 0.75]:
          check=True
          while(check):
               path_x = perc_pos(x_fin,p)
               path_y = perc_pos(y_fin,p)
               pos = df.loc[(df.x==path_x)&(df.y==path_y),['xc_px','yc_px','xc','yc']].values
               if(pos.size==0):
                    p += 0.05
               else:
                    check = False
          pos_hist = torch.cat((pos_hist, torch.Tensor(pos)))
     input_data = torch.cat((input_data, pos_hist.unsqueeze(0)))
     output_data = torch.cat((output_data, torch.Tensor([row[['t0','t1','t2','t3']].values])))

input_data
     

tensor([[[3.1400e+02, 1.3400e+02, 3.5000e-02, 0.0000e+00],
         [1.1343e-01, 3.8783e-01, 1.1343e-01, 6.1217e-01],
         [2.1194e-01, 3.1826e-01, 2.1194e-01, 6.8174e-01],
         [4.0597e-01, 1.6174e-01, 4.0597e-01, 8.3826e-01]],

        [[3.1400e+02, 1.3400e+02, 3.5000e-02, 0.0000e+00],
         [1.1940e-01, 4.1217e-01, 1.1940e-01, 5.8783e-01],
         [2.1194e-01, 3.1826e-01, 2.1194e-01, 6.8174e-01],
         [4.0896e-01, 1.9130e-01, 4.0896e-01, 8.0870e-01]],

        [[3.1400e+02, 1.3400e+02, 3.5000e-02, 0.0000e+00],
         [1.1940e-01, 4.1217e-01, 1.1940e-01, 5.8783e-01],
         [2.6866e-01, 3.2870e-01, 2.6866e-01, 6.7130e-01],
         [4.5672e-01, 2.1739e-01, 4.5672e-01, 7.8261e-01]],

        ...,

        [[3.1400e+02, 1.3400e+02, 3.5000e-02, 0.0000e+00],
         [1.0149e-01, 5.6000e-01, 1.0149e-01, 4.4000e-01],
         [1.0149e-01, 5.6000e-01, 1.0149e-01, 4.4000e-01],
         [1.0149e-01, 5.6000e-01, 1.0149e-01, 4.4000e-01]],

        [[3.1400e+02, 1.3400e+02, 

In [141]:
output_data

tensor([[ 30.1009,  33.9482,  75.0410,  41.0929],
        [ 33.3634,  34.9526,  76.6556,  41.7030],
        [ 36.6366,  34.9526,  76.6556,  41.7030],
        ...,
        [125.0000, 146.0643, 172.2480,  26.1836],
        [110.9638, 152.7918, 172.6161,  19.8243],
        [106.5650, 148.0869, 172.4426,  24.3557]])

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

BATCH_SIZE = 8

X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.2, random_state=42, shuffle=True)

X_test = X_test[4:,:,:]
y_test = y_test[4:,:] # p/ evitar data leakage

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=BATCH_SIZE)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=BATCH_SIZE)

# Separação em conj de validação
#X_train, y_train, X_val, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=42, shuffle=True)

#train_loader

In [164]:
# Define a RNN simples
class ControlRNN(nn.Module):
    def __init__(self, input_size=4, hidden_size=32, num_layers=1, activation='tanh', dropout_rate=0):
        super(ControlRNN, self).__init__()
        # recebe as quatro últimas posições e dá a próxima posição como saída
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, 
                          nonlinearity=activation, dropout=dropout_rate, 
                          batch_first=True)
        # recebe a próxima posição [x, y] e dá os thetas como saída
        #self.dropout = nn.Dropout(p=drop_rate)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        pos, _ = self.rnn(x)
        new_pos = pos[:, -1, :]  # pegamos só a última saída
        thetas = self.fc(new_pos)
        return thetas

In [ ]:
def train_net(model, tloader, vloader, num_epochs, optimizer, lossFunc=nn.MSELoss(), delta=None, patience=None, verbose=2):
    train_losses = []
    test_losses = []
    best_train_score = None
    best_val_score = None
    for e in range(num_epochs):
        train_loss = 0.0 # total loss during single epoch training
        val_loss = 0.0
        model.train()
        for i, (X_batch, y_batch) in enumerate(tloader):
            #X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)

            pred = model(X_batch) # predictions based on batch X_batch
            loss = lossFunc(pred, y_batch)  # calculates the loss function result
            optimizer.zero_grad() # clears x.grad for every parameter x in the optimizer.
            loss.backward() # computes dloss/dx for every parameter x which has requires_grad=True. These are accumulated into x.grad for every parameter x
            optimizer.step() # updates the value of x using the gradient x.grad

            train_loss += loss.item()
            l = np.sqrt(loss.item()) # rmse loss
            train_loss += l # value of loss?
            #print(f'Epoch [{e + 1}/{num_epochs}], Step [{i + 1}/{len(tloader)}], Loss: {l:.4f} ')

        model.eval()
        with torch.no_grad():
            for X_batch, y_batch in vloader:
                #X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
                pred = model(X_batch)
                l = np.sqrt(lossFunc(pred, y_batch).item()) #rmse loss
                val_loss += l

            avg_train_loss = train_loss / len(tloader)
            avg_val_loss = val_loss / len(vloader)
            if(verbose >= 1):
                print(f'Epoch [{e + 1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Eval Loss: {avg_val_loss:.4f}')
        train_losses.append(avg_train_loss)
        test_losses.append(avg_val_loss)

        # Armazenamento do melhor modelo com base no score de validação
        if((best_val_score is None) or (best_val_score > avg_val_loss)):
                    best_val_score = avg_val_loss
                    best_e = e
                    best_model = model
                    if(verbose >= 2):
                        print('best_model updated')

        # Early stopping com base no score de treinamento (não foi usado mas enf)
        if((delta is not None) and (patience is not None)):
            if((best_train_score is None) or (best_train_score-avg_train_loss >= delta)):
                counter = 0
                best_train_score = avg_train_loss
            else:
                counter += 1
                if(counter>=patience):
                    if(verbose >= 2):
                        print("Early Stopping!")
                    break

    return best_model, best_e, (train_losses, test_losses)

In [ ]:
# Dados de exemplo
# Suponha batch_size = 1, seq_len = 4, input_size = 2 (x, y)

# Inicializa a RNN
model = ControlRNN(input_size=4)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)

hidden_size = [300, 100, 50]
num_layers = [1,2]
dropout_rate = [0,0.25,0.75]
learning_rate = [0.0001, 0.001, 0.01]
weight_decay = [0, 0.0005]
activation = ['relu', 'tanh']
optimizer = ['adam', 'sgd']
best_model = None
best_score = None

for hs in hidden_size:
    for nl in num_layers:
        for dr in dropout_rate:
            if(not((dr>0) and (nl==1))):
                for lr in learning_rate:
                    for a in activation:
                        for o in optimizer:
                            #if(o=='lbfgs'):
                            #    model = ControlRNN(input_size=4, hidden_size=hs, 
                            #                        num_layers=nl,
                            #                        activation=a, dropout_rate=dr)
                            #    opt = torch.optim.LBFGS(model.parameters(), lr=lr)
                            #    model, _, scores = train_net(model, train_loader, test_loader, 25, opt, verbose=1)
                            #    score = np.mean(scores[1])
                            #    if(best_score is None or best_score > score):
                            #        print('[Melhor modelo atualizado]')
                            #        print('hs:{}, nl:{}, dr:{}, lr:{}, a:{}, o:{}'.format(hs, nl, dr, lr, a, o))
                            #        best_model = model
                            #        best_score = score
                            #else:
                                for wd in weight_decay:
                                    model = ControlRNN(input_size=4, hidden_size=hs, 
                                                    num_layers=nl,
                                                    activation=a, dropout_rate=dr)
                                    if(o=='adam'):
                                        opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
                                    else:
                                        opt = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=wd)
                                    model, _, scores = train_net(model, train_loader, test_loader, 25, opt, verbose=1)
                                    score = np.mean(scores[1])
                                    if(best_score is None or best_score > score):
                                        print('[Melhor modelo atualizado]')
                                        print('hs:{}, nl:{}, dr:{}, lr:{}, a:{}, o:{}, wd:{}'.format(hs, nl, dr, lr, a, o, wd))
                                        best_model = model
                                        best_score = score
                                

# Teste
predicted = best_model(input_data).detach()
print("Predição:", predicted)

Epoch [1/25], Train Loss: 6880.0900, Eval Loss: 31.2384
Epoch [2/25], Train Loss: 883.3971, Eval Loss: 28.9562
Epoch [3/25], Train Loss: 857.5703, Eval Loss: 28.9589
Epoch [4/25], Train Loss: 857.7728, Eval Loss: 28.9853
Epoch [5/25], Train Loss: 857.6602, Eval Loss: 29.0035
Epoch [6/25], Train Loss: 857.5199, Eval Loss: 29.0146
Epoch [7/25], Train Loss: 857.3793, Eval Loss: 29.0217
Epoch [8/25], Train Loss: 857.2413, Eval Loss: 29.0257
Epoch [9/25], Train Loss: 857.2956, Eval Loss: 29.0288
Epoch [10/25], Train Loss: 857.0484, Eval Loss: 29.0301
Epoch [11/25], Train Loss: 856.9115, Eval Loss: 29.0301
Epoch [12/25], Train Loss: 856.7772, Eval Loss: 29.0294
Epoch [13/25], Train Loss: 856.6415, Eval Loss: 29.0283
Epoch [14/25], Train Loss: 856.4993, Eval Loss: 29.0279
Epoch [15/25], Train Loss: 856.3509, Eval Loss: 29.0257
Epoch [16/25], Train Loss: 856.2066, Eval Loss: 29.0239
Epoch [17/25], Train Loss: 856.0592, Eval Loss: 29.0219
Epoch [18/25], Train Loss: 855.9108, Eval Loss: 29.0194


In [186]:
best_model

ControlRNN(
  (rnn): RNN(4, 300, num_layers=2, batch_first=True)
  (fc): Linear(in_features=300, out_features=4, bias=True)
)

Melhor modelo:

- Número de camadas recorrentes: 2
- Número de neurônios ocultos: 300
- Funcção de ativação: ReLU
- Taxa de dropout: 0
- Ottimizador: Adam
- Learning rate: 0.01
- Weight decay: 0.0005

In [190]:
#with open(, 'wb') as f:
best_model.eval()
torch.save(best_model.state_dict(), 'models/nn_model.pt')

In [ ]:
# Here you can replace pickle with joblib or cloudpickle
#from pickle import load
#with open("models/mlp_base.pkl", "rb") as f:
#    best_mlp = load(f)